In [ ]:
from tifffile import imsave, imread
import glob
from timeit import default_timer as timer
import os.path
import logging
from skimage.io import imshow, imread
import argparse
import io
import json
import os
import re
import csv
import subprocess
from PIL import Image
from math import sqrt, exp, log
from matplotlib import cm
from matplotlib import pyplot as plt
import numpy as np
import micasense.metadata as metadata
from datetime import datetime, timedelta
import pytz
import pandas as pd
import yaml
from glob import glob
from shutil import rmtree, copyfile
import seaborn as sns

In [ ]:
# Constants
exiftool_path = 'c:/exiftool/exiftool.exe'
logging.basicConfig(format='%(levelname)s:%(message)s',
                    level=logging.INFO)  # was WARNING
logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%m/%d/%Y %I:%M:%S %p')



In [ ]:
def fix_time_2_winter_time(datetime_str):
    
    def is_dst(dt,timeZone):
       aware_dt = timeZone.localize(dt)
       return aware_dt.dst() != timedelta(0,0)

    timezone = pytz.timezone('Asia/Jerusalem')
    date_dt = datetime.strptime(datetime_str, '%Y:%m:%d %H:%M:%S')
    if not is_dst(date_dt, timezone): # is winter time?
        date_dt = date_dt - timedelta(hours = 1)

    return date_dt.strftime('%Y:%m:%d %H:%M:%S')    

def extract_raw_image(filename, exiftool_path):
    """
    extracts the thermal image as 2D numpy array in raw data (16 bit information from the camera)
    """

    # exifread can't extract the embedded thermal image, use exiftool instead
    thermal_img_bytes = subprocess.check_output([exiftool_path, "-RawThermalImage", "-b", filename])
    thermal_img_stream = io.BytesIO(thermal_img_bytes)

    thermal_img = Image.open(thermal_img_stream)
    thermal_np = np.array(thermal_img)

    return thermal_np

def extract_meta_data_from_image(filename, exiftool_path):
    """
    extracts necessery meta data for correct conversion of the raw data to temperature (celcius deg)
    Date/Time original time that the flight took place (may need daylight time correction)
    GPS Lat
    GPS Lang
    Camera calibration constants
    Relative Altitude (m)
    """
    
    #meta_json = subprocess.check_output(
    #[exiftool_path, filename, '-GPSLongitude', '-GPSLatitude', '-CreateDate', '-RelativeAltitude', '-PlanckR1', '-PlanckB', '-PlanckF', '-PlanckO', '-PlanckR2', '-j','-Model'])
    #meta = json.loads(meta_json.decode())[0]
    
    meta = metadata.Metadata(filename, exiftoolPath=exiftool_path)
    meta_dict = dict()
    meta_dict = { 'file_name': [filename], 'GPSLon': [meta.get_item('Composite:GPSLongitude')],
                  'GPSLat': [meta.get_item('Composite:GPSLatitude')],
                  'CreateDate_winter_time': [fix_time_2_winter_time(meta.get_item('EXIF:CreateDate'))],
                  'RelativeAltitude': [meta.get_item('XMP:RelativeAltitude')],
                  'PlanckR1': [np.array(meta.get_item('APP1:PlanckR1'))],
                  'PlanckR2': [meta.get_item('APP1:PlanckR2')],
                  'PlanckB': [meta.get_item('APP1:PlanckB')],
                  'PlanckF': [meta.get_item('APP1:PlanckF')],
                  'PlanckO': [meta.get_item('APP1:PlanckO')],
                  'model': [meta.get_item('EXIF:Model')]}

    return meta_dict

def extract_image_and_meta_from_flights(flight_paths):
    
    for flight_path in flight_paths:
        flight_path = os.path.split(flight_path)[0]

        thermal_path = os.path.join(flight_path, 'thermal')
        thermal_tiff_path = os.path.join(flight_path, 'thermal_tiff')
        
        # get a list of images under a flight directory
        for filename in glob(thermal_path + '/*.JPG'):

            # prepare file name for the meta file 
            filename_meta = os.path.splitext(os.path.split(filename)[1])[0] + '_meta.txt'

            # create tiff filename
            tiff_filename = os.path.splitext(os.path.split(filename)[1])[0] + '.tiff'
            tiff_name_path = os.path.join(thermal_tiff_path, tiff_filename)
            meta_name_path = os.path.join(thermal_tiff_path, filename_meta)

            # check if thermal_tiff exists, if it doesn't create a this folder
            if not os.path.isdir(thermal_tiff_path):
                os.mkdir(thermal_tiff_path)

            # check if a corresponding tiff file exists in ./thermal_tiff folder and create it in case it doesn't
            if not os.path.isfile(tiff_name_path):
                logging.info(' extracting tiff image to ' + tiff_filename)
                img = extract_raw_image(filename, exiftool_path)
                imsave(tiff_name_path, img.astype(np.float32)) # Make sure this array is saved as whatever bit depth is necessary. Check what bit depth the np array has    
            else:
                logging.info(tiff_filename + ' exists. aborting extraction...')

            # check meta file does not exist and create it in case it doesn't
            if not os.path.isfile(meta_name_path):
                logging.info(' extracting meta data image to ' + filename_meta)
                met = extract_meta_data_from_image(filename, exiftool_path)
                met_df = pd.DataFrame(met)
                met_df.to_csv(meta_name_path)
            else:
                logging.info(filename_meta + ' exists. aborting extraction...')
    

In [34]:
# in a directory of relfectance images in flight
# for each image in the directory
#    find mean, meadian, std of image
#    extract incoming solar irradiance, timestamp, lat, lon, alt, band_name
# convert pandas dataframe to csv file

import pandas as pd
from enum import Enum
from os import listdir, mkdir
from os.path import isfile, join, basename, split, sep, exists
import micasense.metadata as metadata
import micasense.sequoiautils as msutils
import numpy as np
import logging
import yaml
from glob import glob
import multiprocessing
from threading import Thread
from queue import Queue
from time import sleep
import h5py
import PIL
import subprocess
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from skimage.feature import register_translation
from skimage.transform import resize
from scipy.ndimage.fourier import fourier_shift
mpl.rcParams['figure.dpi'] = 75


logging.basicConfig(format='%(levelname)s:%(message)s',
                    level=logging.INFO)  # was WARNING
logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%m/%d/%Y %I:%M:%S %p')

class Worker(Thread):
    def __init__(self, GPU_id, jobs_queue, results_queue):
        super().__init__()
        self.jobs_queue = jobs_queue
        self.results_queue = results_queue
        self.GPU_id = GPU_id

    def run_unet_segmentation(self, input_img, img_pixelsize_x,
                              img_pixelsize_y, modelfile_path, weightfile_path,
                              iofile_path, remote_path, tiling_x=228,
                              tiling_y=228, gpu_flag='', cleanup=True):
        '''Applies 2D UNET model to the input and returns segmentation & scores


        Parameters
        ----------
        input_img : 2D numpy array (uint8)
            Input image

        img_pixelsize_x : float
            Input image pixelsize in x direction (pixel width)

        img_pixelsize_y : float
            Input image pixelsize in y direction (pixel height)

        modelfile_path : string
            Absolute path to HDF5 unet modelfile

        weightfile_path : string
            Absolute path to matching HDF5 unet weightfile

        iofile_path : string
            Absolute path to location where the temporary caffe
            input/outputfile
            will be created. File must not exist.

        tiling_x : int
            UNET tile size in X direction

        tiling_y : int
            UNET tile size in Y direction

        gpu_flat : string
            GPU mode. Valid strings are
            '' for CPU mode (default)
            'all' for using all available GPUs
            e.g. '0' for using GPU node 0

        cleanup : bool
            If true (default), IO file is deleted after function call.

        Returns
        ---------
        output : dict
            with keys preprocessed_img, scores and segmentation_mask.

        '''

        # fix parameters
        n_inputchannels = 1
        n_iterations = 0

        # run caffe_unet commands
        print('running segmentation using U-net architecture...')
        # assemble sanity check command
        modelfile_path = remote_path + '/' + os.path.split(modelfile_path)[1]
        weightfile_path = remote_path + '/' + os.path.split(weightfile_path)[1]
        command_sanitycheck = []
        command_sanitycheck.append("ssh")
        command_sanitycheck.append("-p")
        command_sanitycheck.append("2222")
        command_sanitycheck.append(remote_info_dict["user_name"] +
                                   '@' + remote_info_dict["IP"])
        command_sanitycheck.append("caffe_unet")
        command_sanitycheck.append("check_model_and_weights_h5")
        command_sanitycheck.append("-model")
        command_sanitycheck.append(modelfile_path)
        command_sanitycheck.append("-weights")
        command_sanitycheck.append(weightfile_path)
        command_sanitycheck.append("-n_channels")
        command_sanitycheck.append(str(n_inputchannels))
        command_sanitycheck.append("-gpu")
        command_sanitycheck.append(str(self.GPU_id))
        # runs command and puts console output to stdout
        # print(command_sanitycheck)
        sanitycheck_proc = subprocess.run(command_sanitycheck,
                                          stdout=subprocess.PIPE)
        # aborts if process failed
        sanitycheck_proc.check_returncode()

        # assemble prediction command
        iofile_path = remote_path + '/' + os.path.split(iofile_path)[1]
        command_predict = []
        command_predict.append("ssh")
        command_predict.append("-p")
        command_predict.append("2222")
        command_predict.append(remote_info_dict["user_name"] +
                               '@' + remote_info_dict["IP"])
        command_predict.append("caffe_unet")
        command_predict.append("tiled_predict")
        command_predict.append("-infileH5")
        command_predict.append(iofile_path)
        command_predict.append("-outfileH5")
        command_predict.append(iofile_path)
        command_predict.append("-model")
        command_predict.append(modelfile_path)
        command_predict.append("-weights")
        command_predict.append(weightfile_path)
        command_predict.append("-iterations")
        command_predict.append(str(n_iterations))
        command_predict.append("-tile_size")
        command_predict.append(str(tiling_x)+'x'+str(tiling_y))
        command_predict.append("-gpu")
        command_predict.append(str(self.GPU_id))
        # if gpu_flag:
        #    command_predict.append("-gpu")
        #    command_predict.append(gpu_flag)
        # runs command
        # print(command_predict)
        with subprocess.Popen(command_predict, stdout=subprocess.PIPE,
                              bufsize=1, universal_newlines=True) as p:
            for line in p.stdout:
                # prints console output
                # print(line, end='')
                pass

    def postprocess_result_from_unet(self, iofile_path, h5ready_img):

        # load results from io file and return
        output_h5 = h5py.File(iofile_path, 'r')
        score = output_h5['score'].value
        output_h5.close()
        # get segmentation mask by taking channel argmax
        segmentation_mask = np.argmax(score, axis=1)

        return dict(preprocessed_img=np.squeeze(h5ready_img),
                    scores=np.squeeze(score),
                    segmentation_mask=np.squeeze(segmentation_mask))

    def save_segmentation_mask(self, result_dict, image_file_name_and_path,
                               segmentation_path):
        print(image_file_name_and_path)
        print(segmentation_path)
        split_path = os.path.split(image_file_name_and_path)
        print(split_path)

        file_name_save_path = os.path.join(segmentation_path,
                                           os.path.splitext(split_path[1])[0]
                                           + '_segmentation.TIF')
        print(file_name_save_path)
        imsave(file_name_save_path,
                  result_dict['segmentation_mask'].astype(np.int8))

    # check if a file exists on the remote computer
    def copy_file_to_remote(self, local_path_and_filename, remote_info_dict,
                            overwrite_flag=False):

        file_name = os.path.split(local_path_and_filename)[1]
        command_sanitycheck = []
        command_sanitycheck.append("ssh")
        command_sanitycheck.append("-p")
        command_sanitycheck.append(remote_info_dict["port"])
        command_sanitycheck.append(remote_info_dict["user_name"] +
                                   "@" + remote_info_dict["IP"])
        command_sanitycheck.append("ls")
        command_sanitycheck.append("data/integration")
        # command_sanitycheck.append(remote_info_dict["path"] + '/'+ file_name)
        # print(command_sanitycheck)
        sanitycheck_proc = subprocess.run(command_sanitycheck,
                                          stdout=subprocess.PIPE)

        if file_name not in sanitycheck_proc.stdout.decode("utf-8"):
            print('uploading ' + file_name + ' to backend server')
            # sftp file to remote
            c = []
            c.append("scp")
            c.append("-P")
            c.append(remote_info_dict["port"])
            c.append(local_path_and_filename)
            c.append(remote_info_dict["user_name"] + "@" +
                     remote_info_dict["IP"] + ":" +
                     remote_info_dict["path"] + '/')
            # print(c)
            subprocess.run(c, stdout=subprocess.PIPE)
            # print(copy_result.returncode)

        if overwrite_flag:
            print('uploading image file to the backend server')
            c = []
            c.append("scp")
            c.append("-P")
            c.append(remote_info_dict["port"])
            c.append(local_path_and_filename)
            c.append(remote_info_dict["user_name"] +
                     "@" + remote_info_dict["IP"] + ":" +
                     remote_info_dict["path"] + '/')
            # print(c)
            subprocess.run(c, stdout=subprocess.PIPE)

    def preprocess_image_file_to_h5(self, input_img, img_pixelsize_x,
                                    img_pixelsize_y, modelfile_path,
                                    remote_info_dict, flight_name):
        # preprocess file to create an h5 image file for UNET segementation
        # fix parameters
        print('preprocessing image for caffe analysis')

        def rescale(size, img, mode='uint8'):
            '''Rescales image to new size, using bilinear interpolation.


            Parameters
            ----------
            size : tuple
                The new image size in pixels, as a 2-tuple: (width, height)

            mode : string
                Datatype to which image is converted before interpolation.
                Valid strings: ['uint8','float32']
            '''
            # resize with bilinear interpolation

            if mode == 'float32':
                # for floating point images:
                img = np.float32(img)
                img_PIL = PIL.Image.fromarray(img, mode='F')
            elif mode == 'uint8':
                # otherwise:
                img_PIL = PIL.Image.fromarray(img)
            else:
                raise(Exception('Invalid rescaling. Use uint8 or float32'))

            return np.array(img_PIL.resize(size, PIL.Image.BILINEAR))

        def normalize(img):
            ''' MIN/MAX-normalizes image to [0,1] range.'''
            # normalize image
            img_min = np.min(img)
            img_max = np.max(img)
            img_centered = img - img_min
            img_range = img_max - img_min
            return img_centered / img_range

        # prepare image rescaling
        # get model resolution (element size) from modelfile
        modelfile_h5 = h5py.File(modelfile_path, 'r')
        modelresolution_y = modelfile_h5['unet_param/element_size_um'][0]
        modelresolution_x = modelfile_h5['unet_param/element_size_um'][1]
        modelfile_h5.close()
        # get input image absolute size
        abs_size_x = input_img.shape[1] * img_pixelsize_x
        abs_size_y = input_img.shape[0] * img_pixelsize_y
        # get rescaled image size in pixel
        rescaled_size_px_x = int(np.round(abs_size_x / modelresolution_x))
        rescaled_size_px_y = int(np.round(abs_size_y / modelresolution_y))
        rescale_size = (rescaled_size_px_x, rescaled_size_px_y)

        # preprocess image and store in IO file

        # normalize image, then rescale
        normalized_img = normalize(input_img)
        rescaled_img = np.float32(rescale(rescale_size,
                                          normalized_img, mode='float32'))
        # prepending singleton dimensions to get the desired blob structure
        h5ready_img = rescaled_img[np.newaxis, np.newaxis, :, :]
        # create caffe IO file
        iofile_path = os.path.split(modelfile_path)[0] + "/temp" + flight_name + str(self.GPU_id) + ".h5"
        if os.path.isfile(iofile_path):
            os.remove(iofile_path)
        iofile_h5 = h5py.File(iofile_path, mode='x')
        # save image blob to hdf5 dataset "/data"
        iofile_h5.create_dataset('data', data=h5ready_img)
        iofile_h5.close()

        return iofile_path, h5ready_img

    # copy from remote back to client
    def copy_file_from_remote_to_local(self, local_path_and_filename,
                                       remote_info_dict, local_img_path_file,
                                       flight_name):

        c = []
        c.append("scp")
        c.append("-P")
        c.append(remote_info_dict["port"])
        c.append(remote_info_dict["user_name"] + "@" + remote_info_dict["IP"]
                 + ":" + remote_info_dict["path"] + "/temp" + flight_name +
                 str(self.GPU_id) + ".h5")
        c.append(os.path.split(local_path_and_filename)[0])
        print('downloading result file ' +
              os.path.split(local_img_path_file)[1] + ' from backend')
        subprocess.run(c, stdout=subprocess.PIPE)

    def run(self):
        print('started worker on GPU' + str(self.GPU_id) + '\n')
        while True:
            try:
                message = self.jobs_queue.get()
                if message is None:
                    break
                local_img_path_file = message["image_name_and_path"]
                self.copy_file_to_remote(local_model_def_path_file,
                                         remote_info_dict)
                self.copy_file_to_remote(local_weight_path_file,
                                         remote_info_dict)
                input_img = imread(local_img_path_file)
                iofile_path, h5ready_img = self.preprocess_image_file_to_h5(input_img,
                                                                            img_pixelsize_x,
                                                                            img_pixelsize_y,
                                                                            local_model_def_path_file,
                                                                            remote_info_dict,
                                                                            message["flight_name"])
                print(iofile_path)
                self.copy_file_to_remote(iofile_path, remote_info_dict,
                                         overwrite_flag=True)
                self.run_unet_segmentation(input_img, img_pixelsize_x,
                                           img_pixelsize_y,
                                           local_model_def_path_file,
                                           local_weight_path_file,
                                           iofile_path,
                                           remote_info_dict["path"],
                                           tiling_x=228, tiling_y=228,
                                           gpu_flag=str(self.GPU_id),
                                           cleanup=True)

                self.copy_file_from_remote_to_local(local_model_def_path_file,
                                                    remote_info_dict,
                                                    local_img_path_file,
                                                    message["flight_name"])
                result_dict = self.postprocess_result_from_unet(iofile_path,
                                                                h5ready_img)
                self.save_segmentation_mask(result_dict,
                                            local_img_path_file,
                                            message["segmentation_path"])
                # io.imshow(result_dict['segmentation_mask']);
                # io.show()
                print('worker ' + str(self.GPU_id) +
                      ' is processing image ' + message["image_name_and_path"]
                      + '\n')
                self.results_queue.put(result_dict)
                self.jobs_queue.task_done()
            except Queue.Empty:
                pass
            print('waiting for jobs...')
            sleep(0.5)

        print('worker on GPU' + str(self.GPU_id) + ' is killed\n')


class VentClass(Thread):
    def __init__(self, number_of_GPUs, flight_path):
        super().__init__()
        self.number_of_GPUs = number_of_GPUs
        self.jobs_queue = Queue()
        self.results_queue = Queue()
        self.worker_threads = []
        self.flight_path = flight_path
        self.local_images_path = os.path.join(flight_path, 'thermal_tiff')
        self.local_segmentation_path = os.path.join(flight_path,
                                                    'segmentation')
        self.job_msg = dict()

    def run(self):
        print('running ventilator class for synchronizing '
              + str(self.number_of_GPUs) + ' GPUs\n')

        print('starting worker threads...\n')
        for i in range(self.number_of_GPUs):
            self.worker_threads.append(Worker(i, self.jobs_queue,
                                              self.results_queue))
            self.worker_threads[i].start()

        print('creating segmentation directory')
        try:
            os.mkdir(self.local_segmentation_path)
        except FileExistsError:
            print('directory ' + self.local_segmentation_path +
                  ' creation failed because it already exists')

        print('creating list of jobs for images..')
        for filename in glob(self.local_images_path + '/*.tiff'):
            seg_filename = os.path.splitext(os.path.split(filename)[1])[0] + '_segmentation.TIF'
            seg_file_path = os.path.join(self.local_segmentation_path, seg_filename)
            if not os.path.isfile(seg_file_path):
                self.job_msg = dict()
                self.job_msg["image_name_and_path"] = filename
                self.job_msg["segmentation_path"] = self.local_segmentation_path
                self.job_msg["flight_name"] = os.path.split(self.flight_path)[1]

                print(filename)
                self.jobs_queue.put(self.job_msg)

        self.jobs_queue.join()
        print('finished segmenting all images\n')

        print('sending the kill signal to running workers\n')
        for i in range(self.number_of_GPUs):
            self.jobs_queue.put(None)

        print('get the results')
        # while not self.results_queue.empty():
        #    print(self.results_queue.get())

        print('finished running ventilator thread\n')

def UNET_anlayze(flight_paths):
    for flight_path in flight_paths:
        flight_path = os.path.split(flight_path)[0]
        #if glob(os.path.join(flight_path, 'analysis') + '/*segmentation_analysis*') == []:
        logging.info(flight_path)
        vent_thread = VentClass(number_of_GPUs=1, flight_path=flight_path)
        vent_thread.start()
        vent_thread.join()
        
def roundTime(dt=None, dateDelta=timedelta(minutes=1)):
    """Round a datetime object to a multiple of a timedelta
    dt : datetime.datetime object, default now.
    dateDelta : timedelta object, we round to a multiple of this, default 1 minute.
    Author: Thierry Husson 2012 - Use it as you want but don't blame me.
            Stijn Nevens 2014 - Changed to use only datetime objects as variables
    """
    roundTo = dateDelta.total_seconds()

    if dt == None : dt = datetime.now()
    seconds = (dt - dt.min).seconds
    # // is a floor division, not a comment on following line:
    rounding = (seconds+roundTo/2) // roundTo * roundTo
    return dt + timedelta(0,rounding-seconds,-dt.microsecond)

# the CORRECT raw to temp function
def raw_to_temp(thermal_np, E, OD, RTemp, ATemp, RH, IRWTemp, IRT, meta_df):
    """
    extracts the thermal image as 2D numpy array with temperatures in oC
    """

    PR1=meta_df["PlanckR1"][0]
    PB=meta_df["PlanckB"][0]
    PF=meta_df["PlanckF"][0]
    PO=meta_df["PlanckO"][0]
    PR2=meta_df["PlanckR2"][0]

    #thermal_np = raw2tempfunc(thermal_np)
    # constants
    ATA1 = 0.006569 #  attenuation for atmosphere without water vap
    ATA2 = 0.01262 #  attenuation for atmosphere without water vap
    ATB1 = -0.002276 # attenuation for water vapor
    ATB2 = -0.00667 # attenuation for water vapor
    ATX = 1.9 # scaling factor atmosphere damping

    # transmission through window (calibrated)
    emiss_wind = 1 - IRT

    # transmission through the air
    
    h2o = (RH / 100) * exp(1.5587 + 0.06939 * (ATemp) - 0.00027816 * (ATemp) ** 2 + 0.00000068455 * (ATemp) ** 3)
    tau = ATX * exp(-sqrt(OD) * (ATA1 + ATB1 * sqrt(h2o))) + (1 - ATX) * exp(
        -sqrt(OD) * (ATA2 + ATB2 * sqrt(h2o)))
    
    # radiance from the environment
    
    raw_refl = PR1 / (PR2 * (exp(PB / (RTemp + 273.15)) - PF)) - PO
    raw_refl_attn = (1 - E) * tau * raw_refl
    raw_atm = PR1 / (PR2 * (exp(PB / (ATemp + 273.15)) - PF)) - PO
    raw_atm_attn = (1 - tau) * raw_atm
    raw_wind = PR1 / (PR2 * (exp(PB / (IRWTemp + 273.15)) - PF)) - PO
    raw_wind_attn = emiss_wind / IRT * raw_wind
    # print(raw_refl_attn, raw_atm_attn, raw_wind_attn, E, tau)
    raw_obj = (thermal_np / IRT - raw_refl_attn - raw_atm_attn - raw_wind_attn)/ E / tau
    temp_celcius = PB / np.log(PR1 / (PR2 * (raw_obj + PO)) + PF) - 273.15
    
    return temp_celcius

def prepare_analysis_dict():
    analysis_dict = dict()
    analysis_dict["file_name"] = []
    analysis_dict["label"] = []
    analysis_dict["Emissivity"] = []
    analysis_dict["Lon"] = []
    analysis_dict["Lat"] = []
    analysis_dict["date_time"] = []
    analysis_dict["relative_altitude_m"] = []
    analysis_dict["model"] = []
    analysis_dict["LWin_W+1m-2"] = []
    analysis_dict["LWout_W+1m-2"] = []
    analysis_dict["SWin_W+1m-2"] = []
    analysis_dict["SWout_W+1m-2"] = []
    analysis_dict["PPFD_umol+1m-2s-1"] = []
    analysis_dict["PPFDr_umol+1m-2s-1"] = []
    analysis_dict["Ta_C"] = []
    analysis_dict["RH_percent"] = []
    analysis_dict["Pa_hPa"] = []
    analysis_dict["Rn_W+1m-2"] = []
    analysis_dict["calculated_reflected_rad_W+1m-2"] = []
    analysis_dict["mean_camera_rad_W+1m-2"] = []
    analysis_dict["std_camera_rad_W+1m-2"] = []
    analysis_dict["median_camera_rad_W+1m-2"] = []
    analysis_dict["max_camera_rad_W+1m-2"] = []
    analysis_dict["min_camera_rad_W+1m-2"] = []
    analysis_dict["coverage_percent"] = []
    analysis_dict["mean_T_apparent_C"] = []
    analysis_dict["std_T_apparent_C"] = []
    analysis_dict["median_T_apparent_C"] = []
    analysis_dict["max_T_apparent_C"] = []
    analysis_dict["min_T_apparent_C"] = []
    
    return analysis_dict


def update_analysis_dict(analysis_dict, img_path_file, object_type, meta_df, img_measurement_time_data,
                        rad_reflected, rad_camera_vector, img_raw, T_apparent):
    # Append data
    analysis_dict["file_name"].append(img_path_file)
    analysis_dict["label"].append(object_type['type'])
    analysis_dict["Emissivity"].append(object_type['emissivity'])
    analysis_dict["Lon"].append(meta_df["GPSLon"][0])
    analysis_dict["Lat"].append(meta_df["GPSLat"][0])
    analysis_dict["date_time"].append(meta_df["CreateDate_winter_time"][0])
    analysis_dict["relative_altitude_m"].append(meta_df["RelativeAltitude"][0])
    analysis_dict["model"].append(meta_df["model"][0])
    analysis_dict["LWin_W+1m-2"].append(img_measurement_time_data['LWin'])
    analysis_dict["LWout_W+1m-2"].append(img_measurement_time_data['LWout'])
    analysis_dict["SWin_W+1m-2"].append(img_measurement_time_data['SWin'])
    analysis_dict["SWout_W+1m-2"].append(img_measurement_time_data['SWout'])
    analysis_dict["PPFD_umol+1m-2s-1"].append(img_measurement_time_data['PPFD'])
    analysis_dict["PPFDr_umol+1m-2s-1"].append(img_measurement_time_data['PPFDr'])
    analysis_dict["Ta_C"].append(img_measurement_time_data['Ta'])
    analysis_dict["RH_percent"].append(img_measurement_time_data['RH'])
    analysis_dict["Pa_hPa"].append(img_measurement_time_data['Pa'])
    analysis_dict["Rn_W+1m-2"].append(img_measurement_time_data['Rn'])
    if np.size(rad_camera_vector) > 3:
        analysis_dict["calculated_reflected_rad_W+1m-2"].append(rad_reflected)
        analysis_dict["mean_camera_rad_W+1m-2"].append(np.mean(rad_camera_vector))
        analysis_dict["std_camera_rad_W+1m-2"].append(np.std(rad_camera_vector))
        analysis_dict["median_camera_rad_W+1m-2"].append(np.median(rad_camera_vector))
        analysis_dict["max_camera_rad_W+1m-2"].append(np.max(rad_camera_vector))
        analysis_dict["min_camera_rad_W+1m-2"].append(np.min(rad_camera_vector))
        analysis_dict["coverage_percent"].append(np.size(rad_camera_vector)/np.size(img_raw)*100)
        analysis_dict["mean_T_apparent_C"].append(np.mean(T_apparent))
        analysis_dict["std_T_apparent_C"].append(np.std(T_apparent))
        analysis_dict["median_T_apparent_C"].append(np.median(T_apparent))
        analysis_dict["max_T_apparent_C"].append(np.max(T_apparent))
        analysis_dict["min_T_apparent_C"].append(np.min(T_apparent))
    else:
        analysis_dict["calculated_reflected_rad_W+1m-2"].append(0)
        analysis_dict["mean_camera_rad_W+1m-2"].append(0)
        analysis_dict["std_camera_rad_W+1m-2"].append(0)
        analysis_dict["median_camera_rad_W+1m-2"].append(0)
        analysis_dict["max_camera_rad_W+1m-2"].append(0)
        analysis_dict["min_camera_rad_W+1m-2"].append(0)
        analysis_dict["coverage_percent"].append(0)
        analysis_dict["mean_T_apparent_C"].append(0)
        analysis_dict["std_T_apparent_C"].append(0)
        analysis_dict["median_T_apparent_C"].append(0)
        analysis_dict["max_T_apparent_C"].append(0)
        analysis_dict["min_T_apparent_C"].append(0)        
    
    return analysis_dict

def extract_segmentation_debug(flight_paths):

    for flight_path in flight_paths:

        # construct relavant path
        flight_path = os.path.split(flight_path)[0]
        thermal_tiff_path = os.path.join(flight_path, 'thermal_tiff')
        seg_path = os.path.join(flight_path, 'segmentation')
        seg_debug_path = os.path.join(flight_path, 'seg_debug')

        logging.info(seg_debug_path)
        if not os.path.exists(seg_debug_path):
            os.mkdir(seg_debug_path)

        for img_path_file in glob(thermal_tiff_path + '/*.tiff'):

            # construct relevant image filenames
            img_filename = os.path.splitext(os.path.split(img_path_file)[1])[0]
            seg_path_filename = os.path.join(seg_path, img_filename + '_segmentation.TIF')
            seg_debug_path_filename = os.path.join(seg_debug_path, img_filename + '_seg_debug.tif')
            
            if not os.path.exists(seg_debug_path_filename):
                img_seg = imread(seg_path_filename)
                img_raw = imread(img_path_file) 
                img_raw = (img_raw - np.min(img_raw)) / (np.max(img_raw)-np.min(img_raw)) * 255

                img_collection = []
                img_collection.append(img_raw.astype(np.int8))
                for object_type in objects:
                    coord = np.where(img_seg == object_type['seg_ID'])

                    # check pixel coordinate by creating a test image
                    test_image = np.zeros(np.shape(img_seg))
                    test_image[coord] = 1
                    img_collection.append(resize(np.multiply(test_image, img_raw), np.divide(np.shape(test_image), 1).astype(np.int)).astype(np.int8))

                img_mont = np.concatenate(img_collection, axis=1)
                logging.info('saving seg_debug ' + seg_debug_path_filename)
                imsave(seg_debug_path_filename, img_mont)
            else:
                logging.info('seg debug file exists. skipping creation')

# Clean analysis
def clean_thermal(flight_paths, clean_seg_debug = False, clean_analysis = False,
                  clean_thermal_tiff = False, clean_segmentation = False):
    
    for flight_path in flight_paths:

        # construct relavant path
        flight_path = os.path.split(flight_path)[0]
        thermal_tiff_path = os.path.join(flight_path, 'thermal_tiff')
        seg_path = os.path.join(flight_path, 'segmentation')
        seg_debug_path = os.path.join(flight_path, 'seg_debug')
        analysis_path = os.path.join(flight_path, 'analysis')

        if clean_seg_debug:
            try:
                rmtree(seg_debug_path)
            except FileNotFoundError:
                logging.info(seg_debug_path + ' directory does not exist')

        if clean_analysis:
            try:
                rmtree(analysis_path)
            except FileNotFoundError:
                logging.info(analysis_path + ' directory does not exist')

        if clean_thermal_tiff:
            try:
                rmtree(thermal_tiff_path)
            except FileNotFoundError:
                logging.info(thermal_tiff_path + ' directory does not exist')
        

        if clean_segmentation:
            try:
                rmtree(seg_path)
            except FileNotFoundError:
                logging.info(seg_path + ' directory does not exist')

def analyze_thermal_segmented_images(measurement_from_str, root_flights, objects_year):
    '''
     pipeline that calculates emited and reflected radiation from a specific object (solar panel or soil)
     inputs: raw image +
             segmented image (panel, soil, shadow) +
             emissivity of objects +
             meta data of raw image +
                   calibration constants for the camera +
                   relative altitude (m) +
                   GPS coordinates +
             measurements 
                   incoming radiation +
                   Air temperature +
                   RH (%) +
     output:
            camera, reflected radiation per object
     What:
        using UNET to segment, 
        get the relevant parameters needed for temperature conversion 
        (which I know now that is 100% correct, assuming FLIR didn't mess up) 
        from the data files (convert incoming radiation to reflected temperature, etc.), 
        get the relevant emmisivity, etc. once I have the temperature, I can convert that 
        to radiation and finish.
    '''
    # constants
    stephan_sigma_const = 5.670374419e-8

    # start of pipeline
    config_file_name = "pipeline_settings.yaml"
    with open(config_file_name, 'r') as stream:
        pipeline_config = yaml.safe_load(stream)

    # read measurement data, replace -9999 with NaNs, change time to index, resample every minute
    # interpolate linearly.
    meas_path_file = pipeline_config[measurement_from_str]
    meas_data_df = pd.read_csv(meas_path_file, skiprows=[1])
    meas_data_df = meas_data_df.where(meas_data_df != -9999) # replace -9999 with NaN
    meas_data_df['TIMESTAMP'] = pd.to_datetime(meas_data_df['TIMESTAMP'], format="%Y-%m-%d %H:%M")
    meas_data_df = meas_data_df.set_index('TIMESTAMP')
    meas_data_min_df = meas_data_df.resample('T').interpolate(method ='time')

    # objects in the image, their segmentation id and emissivites
    objects = pipeline_config[objects_year]

    # read raw and segmented image and meta data
    flight_paths = glob(pipeline_config[root_flights] + '*/')
    for flight_path in flight_paths:

        # construct relavant path
        flight_path = os.path.split(flight_path)[0]
        flight_name = os.path.split(flight_path)[1]
        thermal_tiff_path = os.path.join(flight_path, 'thermal_tiff')
        seg_path = os.path.join(flight_path, 'segmentation')
        analysis_path  = os.path.join(flight_path, 'analysis')
        flight_analysis_path_name = os.path.join(analysis_path, flight_name + '_thermal_analysis.csv')

        if not os.path.exists(flight_analysis_path_name):
            try:
                os.mkdir(analysis_path)
            except FileExistsError:
                logging.info('analysis folder exists, skipping creation')

            analysis_dict = prepare_analysis_dict()

            for img_path_file in glob(thermal_tiff_path + '/*.tiff'):
                # construct 2 different file names and paths
                img_filename = os.path.splitext(os.path.split(img_path_file)[1])[0]
                img_meta_path_file = os.path.join(thermal_tiff_path, img_filename + '_meta.txt')
                img_seg_path_file = os.path.join(seg_path, img_filename + '_segmentation.TIF')

                # load raw and segmented images into memory
                img_raw = imread(img_path_file)
                img_seg = imread(img_seg_path_file)

                # read metadata
                meta_df = pd.read_csv(img_meta_path_file) # access with meta_df['file_name'][0]

                # read measurements at the time when the image was taken
                flight_date_time = datetime.strptime(meta_df['CreateDate_winter_time'][0], '%Y:%m:%d %H:%M:%S')
                img_measurement_time_data = meas_data_min_df.loc[roundTime(flight_date_time, timedelta(minutes=1))]

                # now we have the raw image, segmented image, image meta data, and measurement data
                # we are ready for the calculation of the camera and reflected radiation
                
                sky_emissivity = 1 # CHANGE and UPDATE the sky emissivity 
                
                RTemp = np.power(img_measurement_time_data['LWin']/(sky_emissivity*stephan_sigma_const), 0.25) - 273.15
                for object_type in objects:
                    # calculate temperature in image taking into account all possible factors
                    T_apparent = raw_to_temp(img_raw, object_type['emissivity'], 
                                                  meta_df['RelativeAltitude'][0],
                                                  RTemp, img_measurement_time_data['Ta'], 
                                                  img_measurement_time_data['RH'], 20, 1, meta_df)

                    # transform image to radiation W/m^2
                    camera_received_rad = np.power((T_apparent + 273.15), 4) * \
                                      stephan_sigma_const # removed emissivity - Rafael's request - 08 Nov 2020 - * object_type['emissivity']
                    rad_camera_vector = camera_received_rad[np.where(img_seg == object_type['seg_ID'])]
                    T_apparent_vector = T_apparent[np.where(img_seg == object_type['seg_ID'])]
                    rad_reflected = (1 - object_type['emissivity']) * img_measurement_time_data['LWin']

                    analysis_dict = update_analysis_dict(analysis_dict, img_path_file, object_type, meta_df, 
                                                         img_measurement_time_data, rad_reflected, rad_camera_vector, 
                                                         img_raw, T_apparent_vector)

                    

            # save in the flights loop
            analysis_df = pd.DataFrame(analysis_dict)
            analysis_df.to_csv(flight_analysis_path_name, index = False)

In [37]:
# 2019 thermal dataset
# SEGMENT THE RAW DATA USING U-NET
config_file_name = "pipeline_settings.yaml"
with open(config_file_name, 'r') as stream:
    pipeline_config = yaml.safe_load(stream)
    
# flights root folder
root_flights_folder = pipeline_config['root_thermal_2019_folder']
model_def_config_heading = 'model_def_2019_thermal_folder'
model_config_heading =  'model_2019_thermal_folder'
objects_heading = 'objects_2019'
# read 2019 Ketura measurements
measurement_from_str = 'Ketura_2019_July_measurements'
root_flights = 'root_thermal_2019_folder'

# U-NET server parameters
remote_info_dict = dict()
remote_info_dict["user_name"] = pipeline_config['user_name'] # 'dan1'
remote_info_dict["IP"] = pipeline_config['IP'] # '132.77.34.146'
remote_info_dict["port"] = pipeline_config['port'] # '2222'
remote_info_dict["path"] = pipeline_config['path'] #'/home/dan1/data/integration'
remote_info_dict["gpu"] = pipeline_config['gpu'] # '0'

img_pixelsize_x = 1
img_pixelsize_y = 1

# local UNET model and weights
local_model_def_path_file = pipeline_config[model_def_config_heading]
local_weight_path_file = pipeline_config[model_config_heading]
objects = pipeline_config[objects_heading]

flight_paths = glob(root_flights_folder + '*/')

# The actual procedure calls
extract_image_and_meta_from_flights(flight_paths)
UNET_anlayze(flight_paths)
extract_segmentation_debug(flight_paths)
analyze_thermal_segmented_images(measurement_from_str, root_flights, objects_heading)

INFO:DJI_2033_R.tiff exists. aborting extraction...
INFO:DJI_2033_R_meta.txt exists. aborting extraction...
INFO:DJI_2091_R.tiff exists. aborting extraction...
INFO:DJI_2091_R_meta.txt exists. aborting extraction...
INFO:DJI_2159_R.tiff exists. aborting extraction...
INFO:DJI_2159_R_meta.txt exists. aborting extraction...
INFO:DJI_2177_R.tiff exists. aborting extraction...
INFO:DJI_2177_R_meta.txt exists. aborting extraction...
INFO:DJI_2087_R.tiff exists. aborting extraction...
INFO:DJI_2087_R_meta.txt exists. aborting extraction...
INFO:DJI_2121_R.tiff exists. aborting extraction...
INFO:DJI_2121_R_meta.txt exists. aborting extraction...
INFO:DJI_2037_R.tiff exists. aborting extraction...
INFO:DJI_2037_R_meta.txt exists. aborting extraction...
INFO:DJI_2143_R.tiff exists. aborting extraction...
INFO:DJI_2143_R_meta.txt exists. aborting extraction...
INFO:DJI_2165_R.tiff exists. aborting extraction...
INFO:DJI_2165_R_meta.txt exists. aborting extraction...
INFO:DJI_2115_R.tiff exists.

running ventilator class for synchronizing 1 GPUs

starting worker threads...

started worker on GPU0
creating segmentation directory

directory Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/July 2019/Ketura July 2019 drone/thermal\20072019_1330\segmentation creation failed because it already exists
creating list of jobs for images..
finished segmenting all images

sending the kill signal to running workers

get the results
finished running ventilator thread

worker on GPU0 is killed


INFO:Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/July 2019/Ketura July 2019 drone/thermal\18072019_0815



running ventilator class for synchronizing 1 GPUs

starting worker threads...

started worker on GPU0
creating segmentation directory

directory Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/July 2019/Ketura July 2019 drone/thermal\18072019_0815\segmentation creation failed because it already exists
creating list of jobs for images..


INFO:Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/July 2019/Ketura July 2019 drone/thermal\21072019_0940


...

INFO:Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/July 2019/Ketura July 2019 drone/thermal\22072019_1500


finished segmenting all images

sending the kill signal to running workers

get the results
finished running ventilator thread
worker on GPU0 is killed


running ventilator class for synchronizing 1 GPUs

starting worker threads...

started worker on GPU0
creating segmentation directory

directory Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/July 2019/Ketura July 2019 drone/thermal\22072019_1500\segmentation creation failed because it already exists
creating list of jobs for images..


INFO:Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/July 2019/Ketura July 2019 drone/thermal\20072019_1330\seg_debug
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation

finished segmenting all images

sending the kill signal to running workers

get the resultsworker on GPU0 is killed


finished running ventilator thread



INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skippi

In [38]:
# The actual procedure calls
# 2018 thermal dataset
# extract from dataset a random image from each flight to check UNET NN efficiencies
config_file_name = "pipeline_settings.yaml"
with open(config_file_name, 'r') as stream:
    pipeline_config = yaml.safe_load(stream)
    
# flights root folder

model_def_config_heading = 'model_def_2018_thermal_folder'
model_config_heading =  'model_2018_thermal_folder'
objects_heading = 'objects_2018'
# read 2019 Ketura measurements
measurement_from_str = 'Ketura_2018_October_measurements'
root_flights = 'root_thermal_2018_folder'

# U-NET server parameters
remote_info_dict = dict()
remote_info_dict["user_name"] = pipeline_config['user_name'] # 'dan1'
remote_info_dict["IP"] = pipeline_config['IP'] # '132.77.34.146'
remote_info_dict["port"] = pipeline_config['port'] # '2222'
remote_info_dict["path"] = pipeline_config['path'] #'/home/dan1/data/integration'
remote_info_dict["gpu"] = pipeline_config['gpu'] # '0'

img_pixelsize_x = 1
img_pixelsize_y = 1

# local UNET model and weights
local_model_def_path_file = pipeline_config[model_def_config_heading]
local_weight_path_file = pipeline_config[model_config_heading]
objects = pipeline_config[objects_heading]
root_flights_folder = pipeline_config[root_flights]

# The actual procedure calls
flight_paths = glob(root_flights_folder + '*/')
extract_image_and_meta_from_flights(flight_paths)
UNET_anlayze(flight_paths)
extract_segmentation_debug(flight_paths)
analyze_thermal_segmented_images(measurement_from_str, root_flights, objects_heading)

INFO:DJI_0473_R.tiff exists. aborting extraction...
INFO:DJI_0473_R_meta.txt exists. aborting extraction...
INFO:DJI_0455_R.tiff exists. aborting extraction...
INFO:DJI_0455_R_meta.txt exists. aborting extraction...
INFO:DJI_0371_R.tiff exists. aborting extraction...
INFO:DJI_0371_R_meta.txt exists. aborting extraction...
INFO:DJI_0487_R.tiff exists. aborting extraction...
INFO:DJI_0487_R_meta.txt exists. aborting extraction...
INFO:DJI_0481_R.tiff exists. aborting extraction...
INFO:DJI_0481_R_meta.txt exists. aborting extraction...
INFO:DJI_0475_R.tiff exists. aborting extraction...
INFO:DJI_0475_R_meta.txt exists. aborting extraction...
INFO:DJI_0333_R.tiff exists. aborting extraction...
INFO:DJI_0333_R_meta.txt exists. aborting extraction...
INFO:DJI_0459_R.tiff exists. aborting extraction...
INFO:DJI_0459_R_meta.txt exists. aborting extraction...
INFO:DJI_0461_R.tiff exists. aborting extraction...
INFO:DJI_0461_R_meta.txt exists. aborting extraction...
INFO:DJI_0465_R.tiff exists.

running ventilator class for synchronizing 1 GPUs

starting worker threads...

started worker on GPU0
creating segmentation directory

directory Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/October 2018/2018 thermal images/thermal/Solar Panels flights\20181025_1615\segmentation creation failed because it already exists
creating list of jobs for images..


INFO:Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/October 2018/2018 thermal images/thermal/Solar Panels flights\20181031_1200


finished segmenting all images

sending the kill signal to running workers

get the results
finished running ventilator thread

worker on GPU0 is killed

running ventilator class for synchronizing 1 GPUs

starting worker threads...

started worker on GPU0
creating segmentation directory

directory Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/October 2018/2018 thermal images/thermal/Solar Panels flights\20181031_1200\segmentation creation failed because it already exists
creating list of jobs for images..


INFO:Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/October 2018/2018 thermal images/thermal/Solar Panels flights\20181025_0630


...

INFO:Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/October 2018/2018 thermal images/thermal/Solar Panels flights\20181028_1030


finished segmenting all images

sending the kill signal to running workers

get the results
finished running ventilator thread

worker on GPU0 is killed

running ventilator class for synchronizing 1 GPUs

starting worker threads...

started worker on GPU0
creating segmentation directory

directory Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/October 2018/2018 thermal images/thermal/Solar Panels flights\20181028_1030\segmentation creation failed because it already exists
creating list of jobs for images..


INFO:Z:/Home/GROUP MEMBERS/Rafael/Analyses/Ketura Solar/Drone/October 2018/2018 thermal images/thermal/Solar Panels flights\20181025_1615\seg_debug
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exis

finished segmenting all images

sending the kill signal to running workers

get the results
finished running ventilator thread

worker on GPU0 is killed



INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skipping creation
INFO:seg debug file exists. skippi